# Process US tweet data

Processes and cleans the data, assigns each user/tweets to a county and computes tweet/user indicators

CREATES:
- ID file for all included tweets/users with link to county of tweet and user
- Tweet level data file

In [1]:
################################################################################
# Import modules
################################################################################
import os
import dask
import dask.bag as db
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
import swifter

import pandas as pd
import numpy as np
import statistics as st

import json
import pprint
import geopandas
from shapely.geometry import Polygon
from shapely.geometry import Point
import glob

from matplotlib import pyplot as plt
from collections import Counter

import time
import datetime
import pytz

C:\Users\jakob\.virtualenvs\mapping_mistakes\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
start = time.time()

In [3]:
# Set utilities
pd.options.display.max_colwidth = 100

In [4]:
################################################################################
# Specify paths
################################################################################
os.chdir("../..")
mode = "full" # sample
if mode == "full":
    input_path = "PATH TO FOLDER"
    output_path = "PATH TO FOLDER"
if mode == "sample":
    input_path = "PATH TO FOLDER"
    output_path = "PATH TO FOLDER"
geo_path = "PATH TO FOLDER"

In [5]:
hours = len(glob.glob(input_path))
print(f"Tweets for {hours} hours = {round(hours/24, 1)} days = {round(hours/24/30,1)} months!")

Tweets for 1489 hours = 62.0 days = 2.1 months!


## 1. Create dask dataframe

In [6]:
tweets = db.read_text(input_path) \
            .filter(lambda x: x != '\n') \
            .filter(lambda x: "created_at" in x) \
            .map(lambda x: x.replace("\n", "")) \
            .map(json.loads)

tweet_info = tweets.map(lambda x: [x["id_str"],
                                   x["created_at"],
                                   x["extended_tweet"]["full_text"] if "extended_tweet" in x else x["text"],
                                   x["user"]["id_str"],
                                   x["user"]["followers_count"],
                                   x["user"]["friends_count"],
                                   x["user"]["statuses_count"],
                                   x["user"]["verified"],
                                   x["user"]["description"],
                                   x["user"]["created_at"],
                                   x["place"]["country_code"] if x["place"]!= None else None,
                                   x["coordinates"]["coordinates"] if x["coordinates"] != None else None,
                                   x["place"]["place_type"] if x["place"] != None else None,
                                   x["place"]["name"] if x["place"] != None else None,
                                   x["place"]["bounding_box"]["coordinates"][0] if x["place"] != None else None,
                                   x["is_quote_status"],
                                   x["user"]["is_translator"],
                                   x["source"],
                                   x["lang"]
                                  ])


tweet_dd = tweet_info.to_dataframe(columns = ["id",
                                              "created_at",
                                              "tweet",
                                              "user_id",
                                              "user_followers",
                                              "user_friends",
                                              "user_statuses",
                                              "user_verified",
                                              "user_description",
                                              "user_created_at",
                                              "country_code",
                                              "coordinates",
                                              "place_type",
                                              "place_name",
                                              "bounding_box",
                                              "is_quote_status",
                                              "is_translator",
                                              "source",
                                              "language"
                                             ])



# Keep only English tweets in the US (and drop translators)
tweet_dd = tweet_dd[(tweet_dd["country_code"] == "US") & (tweet_dd["language"] == "en")]
tweet_dd = tweet_dd[tweet_dd["is_translator"]==False]

tweet_dd = tweet_dd.drop(["is_translator", "country_code", "language"], axis=1)

In [7]:
tweet_dd

,id,created_at,tweet,user_id,user_followers,user_friends,user_statuses,user_verified,user_description,user_created_at,coordinates,place_type,place_name,bounding_box,is_quote_status,source
npartitions=1489,,,,,,,,,,,,,,,,
,object,object,object,object,int64,int64,int64,bool,object,object,object,object,object,object,bool,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## 2. Inspect, clean and prepare georeferenced data

### 2.1 Clean and inspect geodata of tweets

In [8]:
place_types = tweet_dd.place_type.value_counts().compute(scheduler='processes')
print(place_types)

city            24005723
admin            4669357
poi               538962
neighborhood       49219
country              237
Name: place_type, dtype: int64


In [9]:
# Check if admin is state or county
tweet_dd[tweet_dd.place_type == "admin"].head() # = state

,id,created_at,tweet,user_id,user_followers,user_friends,user_statuses,user_verified,user_description,user_created_at,coordinates,place_type,place_name,bounding_box,is_quote_status,source
5,1421591480309747719,Sat Jul 31 21:59:55 +0000 2021,"See our latest #Rochester, NY #Engineering job opportunity and click the link in our bio to appl...",24634756,204,162,514,False,"Follow this account for geo-targeted Engineering job tweets in Rochester, NY. Need help? Tweet u...",Mon Mar 16 02:31:01 +0000 2009,"[-77.6109219, 43.16103]",admin,New York,"[[-79.76259, 40.477383], [-79.76259, 45.015851], [-71.777492, 45.015851], [-71.777492, 40.477383]]",False,"<a href=""https://www.careerarc.com"" rel=""nofollow"">CareerArc 2.0</a>"
18,1421591483174510596,Sat Jul 31 21:59:56 +0000 2021,Just posted a photo @ Mt. Mitchell - Elevation 6684 https://t.co/34rJFZBJOf,2482948820,401,1856,46410,False,#blacklivesmatter Haudenosaunee/Seneca/VETERAN/Plant-eater/Indiqueer👬🏳️‍🌈/Sober(11)/(+/u)/Home: ...,Thu May 08 00:28:48 +0000 2014,"[-82.26511002, 35.76587566]",admin,North Carolina,"[[-84.321948, 33.752879], [-84.321948, 36.588118], [-75.40012, 36.588118], [-75.40012, 33.752879]]",False,"<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>"
26,1421591485737222153,Sat Jul 31 21:59:56 +0000 2021,We made it. #Blippiliveshow #Blippi #soexcited #Kadesfirstshow @ York State Fair https://t.co/GN...,236684450,43,81,5072,False,Curiouser and Curiouser,Tue Jan 11 04:10:50 +0000 2011,"[-76.75563812, 39.9574514]",admin,Pennsylvania,"[[-80.519851, 39.719801], [-80.519851, 42.516072], [-74.689517, 42.516072], [-74.689517, 39.7198...",False,"<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>"
59,1421591497028288514,Sat Jul 31 21:59:59 +0000 2021,@richardaeden Shame on all of them.,4893882268,105,295,4225,False,None,Wed Feb 10 02:22:48 +0000 2016,None,admin,Pennsylvania,"[[-80.519851, 39.719801], [-80.519851, 42.516072], [-74.689517, 42.516072], [-74.689517, 39.7198...",False,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>"
74,1421591500958441475,Sat Jul 31 22:00:00 +0000 2021,"Summer in repose. @ Gilgo Beach, New York https://t.co/NAbD0KLQin",24615661,162,102,11827,False,graphic designer. king of the kooks. just trying to survive.,Mon Mar 16 00:22:47 +0000 2009,"[-73.3981, 40.6183]",admin,New York,"[[-79.76259, 40.477383], [-79.76259, 45.015851], [-71.777492, 45.015851], [-71.777492, 40.477383]]",False,"<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>"


In [10]:
# Drop country and state level data
tweet_dd = tweet_dd[(tweet_dd["place_type"].isin(["country", "admin"]) == False) | (tweet_dd["coordinates"]).notnull()]
tweet_dd.count().compute(scheduler='processes')

id                  25072902
created_at          25072902
tweet               25072902
user_id             25072902
user_followers      25072902
user_friends        25072902
user_statuses       25072902
user_verified       25072902
user_description    23308464
user_created_at     25072902
coordinates          2976632
place_type          25072902
place_name          25072902
bounding_box        25072902
is_quote_status     25072902
source              25072902
dtype: int64

### 2.2 Convert tweet data to geodataframe

In [11]:
# Check and clean place types
df = tweet_dd.compute(scheduler='processes')
df.loc[df["coordinates"].notnull(), "place_type"] = "exact"
df.loc[df["coordinates"].notnull(), "place_name"] = np.nan
print(df.place_type.value_counts())
df.place_type.isna().sum() # Should be 0

city            21522995
exact            2976632
poi               538687
neighborhood       34588
Name: place_type, dtype: int64


0

In [12]:
# Convert bounding box and exact coordinates to geometry
df["bounding_box"] = df["bounding_box"].apply(Polygon)
df["coordinates"] = df["coordinates"].apply(lambda x: Point(x) if x != None else np.nan)
df["bounding_box"].head()

0    POLYGON ((-115.384091 36.129459, -115.384091 36.336371, -115.062159 36.336371, -115.062159 36.12...
4    POLYGON ((-74.60247 40.37991, -74.60247 40.40879, -74.548858 40.40879, -74.548858 40.37991, -74....
5    POLYGON ((-79.76259 40.477383, -79.76259 45.015851, -71.777492 45.015851, -71.777492 40.477383, ...
7    POLYGON ((-75.742439 39.409781, -75.742439 39.477614, -75.665128 39.477614, -75.665128 39.409781...
8    POLYGON ((-112.323914 33.29026, -112.323914 33.815465, -111.925439 33.815465, -111.925439 33.290...
Name: bounding_box, dtype: object

In [13]:
# Compute best coordinates (exact coords if available, else centroid of bounding box)
df["best_coords"] = df.apply(lambda x: x["coordinates"] if pd.notna(x["coordinates"])
                                     else x["bounding_box"].centroid, axis=1)
df.drop("coordinates", inplace = True, axis = 1)
df["best_coords"].head()

0    POINT (-115.22312499999998 36.232915)
4              POINT (-74.575664 40.39435)
5             POINT (-77.6109219 43.16103)
7    POINT (-75.70378350000001 39.4436975)
8          POINT (-112.1246765 33.5528625)
Name: best_coords, dtype: object

In [14]:
df.loc[df["place_type"] == "exact", "bounding_box"] = np.nan # Set bounding box to na if exact coordinates are available

In [15]:
# Change projection (to make map look better)
df = df.set_geometry('bounding_box')
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df = df.to_crs("EPSG:5071")
for column in ["best_coords", "bounding_box"]:
    print(df[column].head(2))

0    POINT (-115.22312499999998 36.232915)
4              POINT (-74.575664 40.39435)
Name: best_coords, dtype: object
0    POLYGON ((-1716840.273 1628937.801, -1712146.067 1651631.754, -1684096.318 1645879.236, -1688713...
4    POLYGON ((1785952.173 2131454.925, 1785229.878 2134608.471, 1789626.683 2135616.829, 1790350.756...
Name: bounding_box, dtype: geometry


In [16]:
df = df.set_geometry('best_coords')
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df = df.to_crs("EPSG:5071")
for column in ["best_coords", "bounding_box"]:
    print(df[column].head(2))

0    POINT (-1700451.589 1637392.290)
4     POINT (1787789.942 2133535.676)
Name: best_coords, dtype: geometry
0    POLYGON ((-1716840.273 1628937.801, -1712146.067 1651631.754, -1684096.318 1645879.236, -1688713...
4    POLYGON ((1785952.173 2131454.925, 1785229.878 2134608.471, 1789626.683 2135616.829, 1790350.756...
Name: bounding_box, dtype: geometry


In [17]:
# Import shp-file on us counties and change projection
counties = geopandas.read_file(geo_path + r"\cb_2018_us_county_500k.shp") #!!!!!!!!! CHANGED SHP-file
print(counties.crs)
counties = counties.to_crs("EPSG:5071")
print(counties.crs)

epsg:4269
EPSG:5071


In [18]:
# Mark counties where centroid lies in other county
counties["centroid_bb"] = counties["geometry"].envelope.centroid
counties["centroid_in_mun"] = counties["geometry"].contains(counties["centroid_bb"] )
counties["centroid_in_mun"].value_counts()
print(len(counties.loc[counties["centroid_in_mun"] == False, ["NAME", "GEOID"]]))
counties.loc[counties["centroid_in_mun"] == False, ["NAME", "GEOID"]].head()

77


,NAME,GEOID
26,Aleutians West,02016
28,Nome,02180
29,Yakutat,02282
46,Nevada,06057
73,Fulton,13121


In [19]:
# Check numer of counties with non-unique names (same names in different states)
print(counties.groupby("NAME")["NAME"].transform("count").value_counts())
print(counties.groupby(["STATEFP", "NAME"])["NAME"].transform("count").value_counts()) # Almost unique within states (6 problem counties)

1     1478
2      470
3      261
4      108
5      105
12      96
8       96
6       84
11      66
9       63
10      60
18      54
26      52
24      48
7       35
17      34
31      31
14      28
20      20
16      16
15      15
13      13
Name: NAME, dtype: int64
1    3221
2      12
Name: NAME, dtype: int64


### 2.3 Join with counties dataset

In [20]:
# Note: First performs different types of joins and then compares and checks consistency to perform final join

In [21]:
# Compute area of bounding box for tweets
print("Tweets:")
df["bounding_box_area"] = df["bounding_box"].area/(1000*1000)
df.loc[df["place_type"] == "exact", "bounding_box_area"] = 0
print(df["bounding_box_area"].describe())

# Compute area of bounding box for counties
print("Counties:")
print((counties["geometry"].envelope.area/(1000*1000)).describe())

Tweets:
count    2.507290e+07
mean     5.880555e+02
std      1.041037e+03
min      0.000000e+00
25%      3.264912e+01
50%      1.552927e+02
75%      6.330105e+02
max      5.104632e+03
Name: bounding_box_area, dtype: float64
Counties:
count    3.233000e+03
mean     6.345393e+03
std      4.971142e+04
min      5.092680e-01
25%      1.583265e+03
50%      2.350249e+03
75%      3.719008e+03
max      2.031607e+06
dtype: float64


In [22]:
df.loc[df["bounding_box_area"] > 2000, "place_name"].value_counts() # Bounding boxes seem to be reasonably precise

Los Angeles    867372
Houston        600312
San Antonio    231747
Phoenix        182552
Fort Worth     103604
Castro             61
Name: place_name, dtype: int64

In [23]:
# 1. Join if tweet bounding box is within county polygon (works if county level precision)

In [24]:
# Perform within join for polygons
counties.rename({"geometry": "mun_polygon"}, inplace=True, axis=1)
counties = counties.set_geometry('mun_polygon')
df = df.set_geometry('bounding_box')
df = geopandas.sjoin(df, counties[["GEOID", "NAME", "STATEFP", "mun_polygon"]],
                     how='left', op='within')
df.reset_index(drop=True, inplace=True)
df.drop("index_right", inplace=True, axis=1)

# Rename columns
df.rename({"GEOID": "mun_id_within",
           "NAME": "mun_name_within",
           "STATEFP": "state_id_within",
           'centroid_in_mun': "centroid_in_mun_within"}, axis=1, inplace=True)

In [25]:
# Check results
print(df["mun_id_within"].notna().value_counts()) # Only about 1/4 of observations matched
df.loc[df["mun_id_within"].isna(), ["place_type", "place_name"]].value_counts().head(5)

False    18097506
True      6975396
Name: mun_id_within, dtype: int64


place_type  place_name 
city        Los Angeles    867372
            Houston        600312
            Chicago        513185
            Manhattan      447056
            Brooklyn       367584
dtype: int64

In [26]:
# 2. Join if tweet bounding box contains county polygon (works if only county level precision)

In [27]:
# Perform contains join for polygons
print(len(df))
df = geopandas.sjoin(df, counties[["GEOID", "NAME", "STATEFP", "mun_polygon"]],
                     how='left', op='contains')
df.reset_index(drop=True, inplace=True)
df.drop("index_right", inplace=True, axis=1)
print(len(df)) # Check if it remains the same (if not, some tweet bounding boxes contain several counties)

# Rename columns
df.rename({"GEOID": "mun_id_contains",
           "NAME": "mun_name_contains",
           "STATEFP": "state_id_contains",}, axis=1, inplace=True)

25072902
25072902


In [28]:
# Check nr of tweets that were merged
print(df["mun_id_contains"].notna().value_counts()) # Very few matches (tweet bounding boxes are not usually county level precision)

False    25061848
True        11054
Name: mun_id_contains, dtype: int64


In [29]:
# 3. Join with county that contains centroid of bounding box (problem if mun level precision and centroid not in mun)

In [30]:
# Perform within join for centroids
print(len(df))
df = df.set_geometry("best_coords")
df = geopandas.sjoin(df, counties[["GEOID", "NAME", "STATEFP", "centroid_in_mun", "mun_polygon"]],
                     how='left', op='within')
df.reset_index(drop=True, inplace=True)
df.drop("index_right", inplace=True, axis=1)
len(df)

# Rename columns
df.rename({"GEOID": "mun_id_cwithin",
           "NAME": "mun_name_cwithin",
           "STATEFP": "state_id_cwithin",
           'centroid_in_mun': "centroid_in_mun_cwithin"}, axis=1, inplace=True)
df.geometry.name = "best_coords"

25072902


In [31]:
# Check merge
print(df["mun_id_cwithin"].notna().value_counts()) # Almost all tweets merged
df.loc[df["mun_id_cwithin"].isna(), ["place_type", "place_name"]].value_counts().head(5) # Mostly coastal places

True     25027902
False       45000
Name: mun_id_cwithin, dtype: int64


place_type  place_name  
city        Alameda         5394
            Coronado        3260
            Provincetown    3190
            Malibu          2746
            Key Largo       1739
dtype: int64

In [32]:
# 4. Join with county with nearest bounding box centroid (only correct if county level precision)
counties = counties.set_geometry('centroid_bb')
df = df.set_geometry("best_coords")
print(len(df))
df = geopandas.sjoin_nearest(df, counties[["GEOID",
                                       "NAME",
                                       "STATEFP",
                                       "centroid_in_mun",
                                       "centroid_bb"]].set_geometry('centroid_bb'))
df.reset_index(drop=True, inplace=True)
print(len(df))

# Rename columns
df.rename({"GEOID": "mun_id_nearest",
           "NAME": "mun_name_nearest",
           "STATEFP": "state_id_nearest",
           'centroid_in_mun': "centroid_in_mun_nearest"}, axis=1, inplace=True)

25072902
25072902


In [33]:
# Check merge
print(df["mun_id_nearest"].notna().value_counts()) # Everything merged (by design)

True    25072902
Name: mun_id_nearest, dtype: int64


### Check consistency and perform final merge

In [34]:
# Note: For matches that are deemed correct, the respective county IDs are inserted into column GEOID

In [35]:
df.place_type.value_counts()

city            21522995
exact            2976632
poi               538687
neighborhood       34588
Name: place_type, dtype: int64

In [36]:
# Take (1) within, (2) contains matches for polygons and (3) matches if exact/neighborhood/poi coordinates (safe matches)
df["GEOID"] = df.apply(lambda x:
                       x["mun_id_within"] if pd.notna(x["mun_id_within"])
                       else (x["mun_id_contains"] if pd.notna(x["mun_id_contains"])
                       else (x["mun_id_cwithin"] if x["place_type"] in ["exact", "poi", "neighborhood"]
                             else np.nan)),
                       axis=1)
df["GEOID"].notna().value_counts()

False    15112131
True      9960771
Name: GEOID, dtype: int64

In [37]:
# Take cwithin matches (bb centroids within county) if centroid of matched county is within county
print(df["centroid_in_mun_cwithin"].value_counts())
df["GEOID"] = df.apply(lambda x:
                       x["GEOID"] if pd.notna(x["GEOID"])
                       else (x["mun_id_cwithin"] if x["centroid_in_mun_cwithin"] == True
                             else np.nan),
                       axis=1)
df["GEOID"].notna().value_counts() # Very large share of tweets matched

True     23715267
False     1312635
Name: centroid_in_mun_cwithin, dtype: int64


True     23993451
False     1079451
Name: GEOID, dtype: int64

In [38]:
# Take cwithin matches if cwithin and nearest yield same county
print((df["mun_id_cwithin"] == df["mun_id_nearest"]).value_counts())
df["GEOID"] = df.apply(lambda x:
                       x["GEOID"] if pd.notna(x["GEOID"])
                       else (x["mun_id_cwithin"] if x["mun_id_cwithin"] == x["mun_id_nearest"]
                             else np.nan),
                       axis=1)
df["GEOID"].notna().value_counts() # Almost all tweets matched

True     21497911
False     3574991
dtype: int64


True     24868523
False      204379
Name: GEOID, dtype: int64

In [39]:
# Take cwithin  if place_names is equal to county name for (1) cwithin and (2) nearest if equal to nearest match
df["GEOID"] = df.apply(lambda x:
                       x["GEOID"] if pd.notna(x["GEOID"])
                       else (x["mun_id_cwithin"] if x["mun_name_cwithin"] == x["place_name"]
                       else (x["mun_id_nearest"] if x["mun_name_nearest"] == x["place_name"]
                             else np.nan)),
                       axis=1)
df["GEOID"].notna().value_counts()

True     24911833
False      161069
Name: GEOID, dtype: int64

In [40]:
# Take cwithin if bounding box area is small
df["GEOID"] = df.apply(lambda x:
                       x["GEOID"] if pd.notna(x["GEOID"])
                       else (x["mun_id_cwithin"] if x["bounding_box_area"]<300
                             else np.nan),
                       axis=1)
df["GEOID"].notna().value_counts() # Almost all tweets matched

True     25028053
False       44849
Name: GEOID, dtype: int64

In [41]:
# Inspect remaining tweets
print(df[df["GEOID"].isna()]["mun_name_cwithin"].notna().value_counts())
df.loc[df["GEOID"].isna() & df["mun_name_cwithin"].notna(), ["place_name", "place_type","mun_name_cwithin", "mun_name_nearest"]].value_counts(dropna=False)

False    44849
Name: mun_name_cwithin, dtype: int64


Series([], dtype: int64)

In [42]:
df[df["GEOID"].isna()]["bounding_box_area"].describe()

count    44849.000000
mean        61.598896
std         76.461023
min          0.000000
25%          0.000000
50%         25.495577
75%         82.327763
max        416.040904
Name: bounding_box_area, dtype: float64

In [43]:
# Drop if not assigned to any county
print(f"{df['GEOID'].isna().mean()*100} percent of tweets will be dropped") # 0.2 % of tweets dropped
df = df[df['GEOID'].notna()]

0.1788743879747147 percent of tweets will be dropped


In [44]:
# Add name and drop redundant variables
df = df.merge(counties[["NAME", "GEOID"]],  how="left", on = "GEOID")

In [45]:
df.drop(['mun_id_within', 'mun_name_within', 'state_id_within', 'mun_id_contains',
        'mun_name_contains', 'state_id_contains', 'mun_id_cwithin',
        'mun_name_cwithin', 'state_id_cwithin', 'centroid_in_mun_cwithin',
        'index_right', 'mun_id_nearest', 'mun_name_nearest', 'state_id_nearest',
        'centroid_in_mun_nearest','bounding_box_area'], axis=1, inplace=True)

In [46]:
df.columns

Index(['id', 'created_at', 'tweet', 'user_id', 'user_followers',
       'user_friends', 'user_statuses', 'user_verified', 'user_description',
       'user_created_at', 'place_type', 'place_name', 'bounding_box',
       'is_quote_status', 'source', 'best_coords', 'GEOID', 'NAME'],
      dtype='object')

### Drop duplicates

In [47]:
# Inspect duplicates
df["dupl_id"] = df.duplicated(subset='id', keep = False)
print(df.dupl_id.sum())
dupls = df[df["dupl_id"]].sort_values(by=["user_id", "id"])
#dupls.head(200).to_excel(output_path + r"\inspect\check_dupls.xlsx") # Seem to be real duplicates, not just duplicated ids

11057


In [48]:
# Drop duplicates
print(len(df))
df.drop_duplicates(subset='id', keep='first', inplace=True)
df.drop(["dupl_id"], axis = 1, inplace =True )
print(len(df))

25028053
25021897


## 3. Exlude bots

### 3.1 Inspect source of tweets

In [49]:
df.source.value_counts().to_excel(output_path + r"\inspect\sources.xlsx")
df.source.value_counts().head(10)

OSError: Cannot save file into a non-existent directory: 'H:\analysis_mapping_mistakes\US\01-processed-data\01-tweet_data\inspect'

In [ ]:
# Inspect tweets from instagram
df.loc[df.source == """<a href="http://instagram.com" rel="nofollow">Instagram</a>""", "tweet"].head(500).to_excel(
    output_path + r"\inspect\instagram_examples.xlsx")
df[df.source == """<a href="http://instagram.com" rel="nofollow">Instagram</a>"""].head() # Does not look like bots

In [ ]:
df[df.source == """<a href="https://www.careerarc.com" rel="nofollow">CareerArc 2.0</a>"""].head()

In [ ]:
# Add column to identify tweets that are not posted through a third party API (???)
keep_sources = ["""<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>""",
                """<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>""",
                """<a href="http://instagram.com" rel="nofollow">Instagram</a>""",
                """<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>"""]

df["nobot"] = False
df.loc[df["source"].isin(keep_sources), "nobot"] = True
df[df["nobot"]].sample(10)

In [ ]:
df["nobot"].value_counts()

### 3.2. Inspect users with high number of statuses

In [ ]:
df.groupby("nobot")["user_statuses"].describe()

In [ ]:
df.loc[(df["user_statuses"] > 200000) & (df["nobot"]),
             ["NAME", "tweet", "user_id", "user_description", "source"]].drop_duplicates(subset="user_id").head(10) # Does not look like bots

### 3.3. Exclude bots using source of tweets

In [ ]:
# Keep only: iPhone, Android, Instagram, iPad
df = df[df["nobot"]]
df.drop("nobot", axis=1, inplace=True)

## 4. Compute local time for each tweet

In [ ]:
# Inspect format (UCT time)
df["created_at"].head()

In [ ]:
# Compute time zone based on coordinates
from tzwhere import tzwhere # Not sure why, but needs to be imported here
tzwhere = tzwhere.tzwhere(forceTZ=True)
df['lon'] = df.to_crs("epsg:4326").best_coords.x
df['lat'] = df.to_crs("epsg:4326").best_coords.y
df["timezone"] = df.apply(lambda x: tzwhere.tzNameAt(x["lat"], x["lon"], forceTZ=True), axis = 1)
print(sum(df["timezone"].isna())) # Should be none
df["timezone"].value_counts().head()

In [ ]:
# Compute local time for each tweet
df["created_at"] = pd.to_datetime(df["created_at"], format = '%a %b %d %H:%M:%S +0000 %Y', utc=True)
def tz_func(x):
    return x.dt.tz_convert(x.name).dt.tz_localize(tz=None)
df["local_time"] = df.groupby("timezone")["created_at"].transform(tz_func)
df[["created_at", "timezone", "local_time"]].head()

In [ ]:
# Create separate variables for day and time
df["hour"] = df["local_time"].dt.hour
df["day"] = df["local_time"].dt.weekday # 0 = Monday, 6 = Sunday
print(df["hour"].value_counts())
print(df["day"].value_counts())

In [ ]:
# Define weekdays
df["weekday"] = df["day"] < 5
df["weekday"].value_counts()

In [ ]:
# Distinguish between free time and worktime
df["workhours"] = (df["weekday"]) & (df["hour"] >= 9) & (df["hour"] < 17)
df["workhours"].value_counts()

## 5. Handle users that tweet from different counties

In [ ]:
# Inspect nr of counties users tweet from
counties_per_user = df.groupby(["user_id", "GEOID"]).count()
counties_per_user = counties_per_user.groupby("user_id")["lon"].count().sort_values(ascending=False)
print(counties_per_user.describe())
print(len(counties_per_user[counties_per_user>5]))
print(len(counties_per_user[counties_per_user>10]))
print(len(counties_per_user[counties_per_user>20]))
print(len(counties_per_user[counties_per_user>50]))
print(len(counties_per_user[counties_per_user>100]))
print(len(counties_per_user[counties_per_user==1]))
counties_per_user.value_counts(normalize=True).head(10)

In [ ]:
# Inspect users who tweet from many counties
df["nr_counties"] = df.groupby("user_id")["GEOID"].transform("nunique") # add variable for nr or counties each user tweets from
if mode == "full":
    df.loc[df["nr_counties"] > 50, ["NAME", "tweet", "user_description"]].sample(20)  # Looks like normal users (e.g. truck drivers)

In [ ]:
# Get main county/counties for each user
df["main_county_all"] = df.groupby("user_id")["GEOID"].transform(lambda x: ",".join(st.multimode(x)))
df["nr_main_counties"] = df.groupby("user_id")["GEOID"].transform(lambda x: len(st.multimode(x)))
print(df["main_county_all"].value_counts().head())
print(df["nr_main_counties"].value_counts())
print(df["nr_main_counties"].value_counts(normalize=True))

In [ ]:
# Get main county/counties for each user excluding tweets during workhours or weekends (when people might not be at home)
df["county_home"] = df["GEOID"]*1
df["county_home"].loc[(df["weekday"] == False) | (df["workhours"] == True)] = np.nan
df["main_county_home"] = df.groupby("user_id")["county_home"].transform(lambda x: ",".join(
    st.multimode(x[x.notna()])))

df["nr_home_counties"] = df.groupby("user_id")["county_home"].transform(lambda x: len(st.multimode(x[x.notna()])))
print(df["main_county_home"].value_counts().head())
print(df["nr_home_counties"].value_counts())
print(df["nr_home_counties"].value_counts(normalize=True))

In [ ]:
df[["main_county_all", "main_county_home", "nr_counties", "nr_home_counties"]].head(20)

In [ ]:
# Define var "main_county": main county if unambiguous, else main home county (based on time) if unambiguous, else NaN
df.reset_index(drop=True, inplace=True)
df["main_county"] = df.apply(lambda x:
                                     x["main_county_all"] if x["nr_main_counties"] == 1
                                     else (x["main_county_home"] if x["nr_home_counties"] == 1
                                           else np.nan),
                                     axis=1)
df["main_county"].head()

In [ ]:
# Inspect number of ambiguous cases
print(df["main_county"].isna().sum())
print(df["main_county"].isna().mean())

In [ ]:
df[["GEOID", "user_id", "main_county_all", "main_county"]].head()

In [ ]:
# Get county-user for main county
print("Total nr of counties: ", len(counties["GEOID"].unique()))
print("Nr of counties with at least one tweet: ", len(df["GEOID"].unique()))
print("Nr of counties with at least one tweet as main county: ", len(df["main_county"].unique()))

In [ ]:
df.groupby("main_county")["user_id"].count().describe()

## 6. Finalize and export dataset and ID file

### 6.1. Clean and add columns with additional information

In [ ]:
df.columns

In [ ]:
# Drop redundant columns
df.drop(['main_county_all',
         'nr_main_counties',
         'county_home',
         'main_county_home',
         'nr_home_counties'],
        inplace = True,
        axis = 1)

In [ ]:
# Place type and coordinates
df.drop(['best_coords'],
        inplace = True, axis = 1)
df["place_type"].value_counts()

In [ ]:
# County name and ID
df.rename({"GEOID":"county_id_tweet",
           "NAME" :"county_name_tweet",
           "main_county": "county_id_user"},
          axis=1, inplace=True)
df = df.merge(counties[["GEOID", "NAME"]],
              left_on="county_id_user",
              right_on="GEOID",
              how="inner")
df.drop("GEOID", inplace=True, axis=1)
df.rename({"NAME": "county_name_user"}, axis=1, inplace=True)

In [ ]:
# iPhone users
df["iphone"] = df["source"].isin(["""<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>""",
                                  """<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>"""])
df["iphone"].value_counts()

In [ ]:
# Instagram retweets
df["insta"] = df["source"] == """<a href="http://instagram.com" rel="nofollow">Instagram</a>"""
print(df["insta"].value_counts(normalize=True))

# Check Instagram tweets with default retweet text
insta = df[df["source"] == """<a href="http://instagram.com" rel="nofollow">Instagram</a>"""]
print(insta["tweet"].str.startswith("Just posted a photo").value_counts(normalize=True))

# Exclude Instagram tweets with default retweet text
df = df.loc[~((df["source"] == """<a href="http://instagram.com" rel="nofollow">Instagram</a>""") &
              (df["tweet"].str.startswith("Just posted a photo"))), :]

In [ ]:
# Length of tweet and user description
df["tweet_length"] = df["tweet"].str.len()
df["user_length"] = df["user_description"].str.len()
df.loc[df["user_length"].isna(), "user_length"] = 0
print(df["tweet_length"].describe())
print(df["user_length"].describe())

In [ ]:
# Mobility of users
df.rename({"nr_counties":"user_nr_counties"}, inplace=True, axis=1)
df["user_nr_counties"].describe()

In [ ]:
# Order columns
move_cols = ['county_id_tweet', 'county_name_tweet', 'county_id_user', 'county_name_user']
cols  = move_cols + [col for col in df.columns if col not in move_cols]
df = df[cols]
df.columns

In [ ]:
# Drop tweets in US territories (Puerto Rico, Virgin Islands)
df["state_id_user"] = df["county_id_user"].str[:2]

In [ ]:
df.loc[df["state_id_user"].astype(int) >=60, "state_id_user"].value_counts()

In [ ]:
df = df[df["state_id_user"].astype(int)<60] # Drop if not in a US state (but only in territories)

In [ ]:
print(df["state_id_user"].value_counts())
print(len(df["state_id_user"].value_counts()))
df.drop(columns="state_id_user", inplace=True)

In [ ]:
print(len(df))
df.head()

### 6.2. Export files


In [ ]:
# Export ID file
df[["id", "user_id", "county_id_tweet", "county_id_user"]].to_csv(output_path + r"\ids.csv",
                                                                  encoding="utf-8",
                                                                  sep=";",
                                                                  index=False)

In [ ]:
# Export full dataset (takes long)
df.to_csv(output_path + r"\tweet_data.csv",
          encoding="utf-8",
          sep=";",
          index=False)

In [ ]:
# Print nr of users and tweets in final dataset
print(len(df))
print(df["user_id"].nunique())


In [ ]:
df["place_type"].value_counts(normalize=True)